In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pneumo import dataset
import os
import glob


In [3]:
IMG_PATH = "/home/wilmer-linux/Projects/data/pneumo_s1/dicom-images-train/"
LABEL_PATH = os.path.join(IMG_PATH, '../train-rle.csv')

fns = glob.glob(os.path.join(IMG_PATH, "**/*.dcm"), recursive=True)

ds = dataset.PneumoDataset(LABEL_PATH, fns)

In [7]:
from torch.utils.data import DataLoader

In [10]:
bs = 32

loader = DataLoader(ds, batch_size=32, shuffle=True, num_workers=4)

In [17]:
batch = next(iter(loader))

In [19]:
batch['target'].shape

torch.Size([32, 1024, 1024])

In [16]:
ds[0]

{'img': array([[219, 216, 216, ...,   6,   4,   1],
        [215, 213, 212, ...,   6,   4,   1],
        [213, 210, 210, ...,   6,   4,   1],
        ...,
        [216, 216, 215, ...,  15,   8,   0],
        [217, 216, 215, ...,  17,  10,   1],
        [217, 215, 214, ...,  18,  11,   2]], dtype=uint8),
 'age': '67',
 'sex': 'M',
 'view_position': 'PA',
 'target': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])}